In [42]:
#loading the unprocessed data
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import nltk
from google.colab import drive
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import FeatureUnion
import re
import string
from scipy.sparse import hstack, csr_matrix
from sklearn.model_selection import train_test_split
from sklearn.linear_model import RidgeClassifier
from sklearn.metrics import accuracy_score
from sklearn import model_selection, naive_bayes, svm
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
import os

In [43]:
#drive.mount('/content/drive')
#os.chdir("/content/drive/My Drive/")

In [44]:
  project_data = pd.read_csv('Text classification/craiglist_1.2k_appliances_data.csv')
  project_data.head()

,link,Title,Description,Price,Condition
0,https://tippecanoe.craigslist.org/app/d/montic...,maytag electric dryer - $150,"maytag electric steam dryer, $150. obo. i have...",$150,NaN
1,https://tippecanoe.craigslist.org/app/d/fisher...,Platinum GE Washer and Dryer - $495 (Fishers),We are selling our like new set of Platinum GE...,$495,like new
2,https://tippecanoe.craigslist.org/ppd/d/washer...,WASHERS*WASHERS*WASHERS* $$$ SAVE $$$* DRYERS*...,SAMSUNG SUPER Capacity Multi Cycle Washer (STA...,NaN,NaN
3,https://tippecanoe.craigslist.org/app/d/lafaye...,*ESTATE SALE* Hotpoint Electric Range - $40 (L...,*****ESTATE SALE*****Hotpoint Electric RangeMo...,$40,good
4,https://tippecanoe.craigslist.org/app/d/lafaye...,*PRICE REDUCED / ESTATE SALE* Maytag Dishwashe...,*****ESTATE SALE*****Maytag DishwasherModel WU300,$15,good


In [45]:
project_data['Description'] = project_data['Description'].fillna('Unknown')

In [46]:
project_data['Description_char'] = project_data['Description'].values

In [47]:
project_data['ex_flag'] = project_data['Description'].str.contains('excellent|Excellent').astype(int)
project_data['new_flag'] = project_data['Description'].str.contains('new|New').astype(int)
project_data['go_flag'] = project_data['Description'].str.contains('good|Good').astype(int)

In [48]:
filtered_data = project_data[~project_data['Condition'].isna()]
filtered_data = filtered_data[~project_data['Price'].isna()]
filtered_data['Price'] = filtered_data['Price'].str.replace('$', '').str.replace(',', '').astype('float')
filtered_data['summary_size']     = filtered_data['Description'].apply(lambda x: len(x))
filtered_data['words_in_summary'] = filtered_data['Description'].apply(lambda x: len(x.split()))
filtered_data['Condition'] = filtered_data['Condition'].map({'excellent': 1, 'new': 1, 'good':0, 'like new':0, 'fair':0 })
filtered_data = filtered_data[~filtered_data['Condition'].isna()]

filtered_data.shape

(798, 11)

In [49]:
filtered_data['Condition'].value_counts(normalize=True)

1.0    0.545113
0.0    0.454887
Name: Condition, dtype: float64

In [50]:
default_preprocessor = CountVectorizer().build_preprocessor()
def build_text_preprocessor(field):
  field_idx = list(filtered_data.columns).index(field)
  return lambda x: default_preprocessor(x[field_idx])

re_tok = re.compile(f'([{string.punctuation}''])')
def tokenize(s):
  return re_tok.sub(r' ', s).split(' ')

In [51]:
text_vectorizer = FeatureUnion([
                                ('Description', TfidfVectorizer(ngram_range=(1,3), tokenizer=tokenize, min_df=0.01, max_df=0.9,
                                                                strip_accents='unicode',use_idf=1, smooth_idf=1, sublinear_tf=1,
                                                                preprocessor = build_text_preprocessor('Description'))),
                                ('Description_char', TfidfVectorizer(ngram_range=(1,4), tokenizer=tokenize, analyzer='char', 
                                                                     stop_words='english', strip_accents='unicode',
                                                                     max_features=10000,
                                                                preprocessor = build_text_preprocessor('Description_char'))),
                                ('Title', TfidfVectorizer(ngram_range=(1,3), tokenizer=tokenize, min_df=0.01, max_df=0.9,
                                                                strip_accents='unicode',use_idf=1, smooth_idf=1, sublinear_tf=1,
                                                                preprocessor = build_text_preprocessor('Title')))
                              ])
train_tfidf = text_vectorizer.fit_transform(filtered_data.values)

In [52]:
numeric_features = csr_matrix(filtered_data[[ 'summary_size', 'words_in_summary', 'Price', 'ex_flag', 'new_flag', 'go_flag']].values)
train_idf = hstack((train_tfidf, numeric_features)).toarray()

In [53]:
y = filtered_data['Condition']

In [54]:
X_train, X_valid, y_train, y_valid = train_test_split(train_idf, y, test_size=0.2, random_state=1, stratify = y )

In [55]:
classifier = RidgeClassifier(alpha=.0001)
classifier.fit(X_train, y_train)
pred_tfidf = classifier.decision_function(X_valid)
probs_tfidf = np.exp(pred_tfidf)
pred_tfidf = np.where(probs_tfidf>0.90, 1, 0)
accuracy_val = accuracy_score(y_valid, pred_tfidf)
accuracy_val

0.7125

In [56]:
filtered_data['Condition'].value_counts(normalize=True)

1.0    0.545113
0.0    0.454887
Name: Condition, dtype: float64

In [57]:
gnb = GaussianNB()
y_pred = gnb.fit(X_train, y_train).predict(X_valid)
accuracy_gb = accuracy_score(y_valid, y_pred)
print("Gaussian Naive Bayes Accuracy Score -> ",accuracy_gb*100)

Gaussian Naive Bayes Accuracy Score ->  66.25


In [58]:
Naive = naive_bayes.MultinomialNB()
Naive.fit(X_train,y_train)
predictions_NB = Naive.predict(X_valid)
print("Naive Bayes Accuracy Score -> ",accuracy_score(predictions_NB, y_valid)*100)

Naive Bayes Accuracy Score ->  60.62499999999999


In [ ]:
# SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
# SVM.fit(X_train,y_train)
# predictions_SVM = SVM.predict(X_valid)# Use accuracy_score function to get the accuracy
# print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, y_valid)*100)

In [59]:
#Random Forest Classifier
random_forest = RandomForestClassifier(n_estimators=50)
y_pred_rf = random_forest.fit(X_train,y_train).predict(X_valid)
accuracy_rf = accuracy_score(y_pred_rf, y_valid)
print("Random Forest Accuracy Score -> ",accuracy_rf*100)

Random Forest Accuracy Score ->  70.0


In [60]:

import pandas as pd
from keras.preprocessing import text,sequence
import numpy as np
from keras.layers import GRU, Conv1D,CuDNNGRU
from keras.layers import Bidirectional, GlobalAveragePooling1D, GlobalMaxPooling1D, concatenate, SpatialDropout1D
from keras.callbacks import Callback,EarlyStopping
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, Embedding, Dropout, Conv1D
from keras.models import Model, load_model
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score,roc_auc_score,f1_score
import re,string
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import FeatureUnion
from scipy import sparse
from scipy.sparse import hstack, csr_matrix
from keras.models import model_from_json
import warnings
warnings.filterwarnings("ignore")

In [61]:
filtered_data.head()

,link,Title,Description,Price,Condition,Description_char,ex_flag,new_flag,go_flag,summary_size,words_in_summary
1,https://tippecanoe.craigslist.org/app/d/fisher...,Platinum GE Washer and Dryer - $495 (Fishers),We are selling our like new set of Platinum GE...,495.0,0.0,We are selling our like new set of Platinum GE...,0,1,0,472,83
3,https://tippecanoe.craigslist.org/app/d/lafaye...,*ESTATE SALE* Hotpoint Electric Range - $40 (L...,*****ESTATE SALE*****Hotpoint Electric RangeMo...,40.0,0.0,*****ESTATE SALE*****Hotpoint Electric RangeMo...,0,0,0,151,21
4,https://tippecanoe.craigslist.org/app/d/lafaye...,*PRICE REDUCED / ESTATE SALE* Maytag Dishwashe...,*****ESTATE SALE*****Maytag DishwasherModel WU300,15.0,0.0,*****ESTATE SALE*****Maytag DishwasherModel WU300,0,0,0,49,4
5,https://tippecanoe.craigslist.org/app/d/fisher...,Roper Washer and Dryer - $325 (Fishers),Whirlpool Roper Washer and Electric Dryer. Abo...,325.0,1.0,Whirlpool Roper Washer and Electric Dryer. Abo...,0,0,0,209,34
6,https://tippecanoe.craigslist.org/app/d/lafaye...,Front load washer - $250 (Lafayette downtown),I have a Crosley front load washer that was us...,250.0,0.0,I have a Crosley front load washer that was us...,0,1,0,119,23


In [62]:
# !wget https://dl.fbaipublicfiles.com/fasttext/vectors-english/crawl-300d-2M.vec.zip

In [63]:
# !unzip Text\ classification/crawl-300d-2M.vec.zip

In [64]:
embed_size = 300
max_features = 6500
maxlen = 350
tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(filtered_data['Description']))
EMBEDDING_FILE = 'Text classification/crawl-300d-2M.vec'
print('loading embedding file')
def get_coefs(word, *arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.rstrip().rsplit(' ')) for o in open(EMBEDDING_FILE,encoding="utf8"))
print('Done loading embedding file')
word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix = np.zeros((nb_words, embed_size))
for word, i in word_index.items():
  if i >= max_features: continue
  embedding_vector = embeddings_index.get(word)
  if embedding_vector is not None: embedding_matrix[i] = embedding_vector

loading embedding file
Done loading embedding file


In [65]:
def build_model(body_flag,body_flag_test, X_train,Y_train,X_valid,Y_valid,check_point,early_stop,file_path, lr = 0.0, lr_d = 0.0, units = 0, dr = 0.0):
    body_text_flag = body_flag
    inp = Input(shape = (maxlen,))
    print(nb_words)
    x = Embedding(max_features , embed_size, weights = [embedding_matrix], trainable = False)(inp)

    x = SpatialDropout1D(dr)(x)

    x = Bidirectional(GRU(64, return_sequences = True))(x)
    x = Conv1D(64, kernel_size = 1, padding = "valid", kernel_initializer = "he_uniform")(x)
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    
    # body_text_flag = Input(shape=[body_text_flag.shape[1]], name="body_text_flag")
    x = concatenate([avg_pool, max_pool])
    x = Dense(1, activation = "sigmoid")(x)
    model = Model(inputs = inp, outputs = x)
    ra_val = RocAucEvaluation(validation_data=(X_valid, Y_valid), interval = 1)
    model.compile(loss = "binary_crossentropy", optimizer = Adam(lr = lr, decay = lr_d), metrics = ["accuracy"])
    history = model.fit(X_train , Y_train, batch_size = 64, epochs = 15, validation_data = (X_valid, Y_valid), 
                        verbose = 1, callbacks = [ra_val, check_point, early_stop])
    model = load_model(file_path)
    return model

class RocAucEvaluation(Callback):
    def __init__(self, validation_data=(), interval=1):
        super(Callback, self).__init__()

        self.interval = interval
        self.X_val, self.y_val = validation_data

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            y_pred = self.model.predict(self.X_val, verbose=0)
            score = roc_auc_score(self.y_val, y_pred)
            print("\n ROC-AUC - epoch: %d - score: %.6f \n" % (epoch+1, score))

In [66]:
def build_model(body_flag,body_flag_test, X_train,Y_train,X_valid,Y_valid,check_point,early_stop,file_path, lr = 0.0, lr_d = 0.0, units = 0, dr = 0.0):
    body_text_flag = body_flag
    inp = Input(shape = (maxlen,))
    x = Embedding(max_features, embed_size, weights = [embedding_matrix], trainable = False)(inp)
    x = SpatialDropout1D(dr)(x)

    x = Bidirectional(GRU(units, return_sequences = True))(x)
    x = Conv1D(64, kernel_size = 2, padding = "valid", kernel_initializer = "he_uniform")(x)
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    body_text_flag = Input(shape=[body_text_flag.shape[1]], name="body_text_flag")
    x = concatenate([avg_pool, max_pool, body_text_flag])
    x = Dense(1, activation = "sigmoid")(x)
    model = Model(inputs = [inp, body_text_flag], outputs = x)
    ra_val = RocAucEvaluation(validation_data=([X_valid,body_flag_test], Y_valid), interval = 1)
    model.compile(loss = "binary_crossentropy", optimizer = Adam(lr = lr, decay = lr_d), metrics = ["accuracy"])
    history = model.fit([X_train,body_flag] , Y_train, batch_size = 64, epochs = 15, validation_data = ([X_valid,body_flag_test], Y_valid), 
                        verbose = 1, callbacks = [ra_val, check_point, early_stop])
    model = load_model(file_path)
    return model

class RocAucEvaluation(Callback):
    def __init__(self, validation_data=(), interval=1):
        super(Callback, self).__init__()

        self.interval = interval
        self.X_val, self.y_val = validation_data

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            y_pred = self.model.predict(self.X_val, verbose=0)
            score = roc_auc_score(self.y_val, y_pred)
            print("\n ROC-AUC - epoch: %d - score: %.6f \n" % (epoch+1, score))

In [67]:
X_train, X_valid, Y_train, Y_valid = train_test_split(filtered_data.drop(['Condition'], axis=1), y, test_size = 0.2, random_state =1,stratify=y)
raw_text_train = 'Title' + X_train["Title"].astype(str) + ' ' + X_train["Description"]
raw_text_valid = 'Title' + X_valid["Title"].astype(str) + ' ' + X_valid["Description"]

tk = text.Tokenizer(num_words = max_features, lower = True)
tk.fit_on_texts(raw_text_train)
X_train["text_seq"] = tk.texts_to_sequences(raw_text_train)
X_valid["text_seq"] = tk.texts_to_sequences(raw_text_valid)

print('padding')
raw_text_train = pad_sequences(X_train.text_seq, maxlen = maxlen)
raw_text_test = pad_sequences(X_valid.text_seq, maxlen = maxlen)


file_path = "best_model.hdf5"
check_point = ModelCheckpoint(file_path, monitor = "val_acc", verbose = 1,
                      save_best_only = True, mode = "max")
early_stop = EarlyStopping(monitor = "val_acc", mode = "max", patience = 5)
test_flag = X_train[['ex_flag', 'new_flag', 'go_flag']].values
body_flag_test = X_valid[['ex_flag', 'new_flag', 'go_flag']].values
model = build_model(test_flag,body_flag_test,raw_text_train,Y_train,raw_text_test,Y_valid,check_point,early_stop,file_path,lr = 1e-3, lr_d = 0, units = 64, dr = 0.2)

model.load_weights(file_path)
predictions = model.predict([raw_text_test,body_flag_test]).reshape((raw_text_test.shape[0],))

# pred = loaded_model.predict(X_valid, batch_size = 1024, verbose = 1)
pred = np.where(predictions>=0.5,1,0)
score = pd.DataFrame(pred)
# print(Y_valid,score.astype(int))
accuracy_val = accuracy_score(Y_valid, score.values)
print('NN OOF accuracy: {}'.format(accuracy_val))

padding
Epoch 1/15
10/10 [==============================] - ETA: 0s - loss: 0.6888 - accuracy: 0.5235
 ROC-AUC - epoch: 1 - score: 0.729177 

10/10 [==============================] - 14s 1s/step - loss: 0.6888 - accuracy: 0.5235 - val_loss: 0.6582 - val_accuracy: 0.6125
Epoch 2/15
10/10 [==============================] - ETA: 0s - loss: 0.6334 - accuracy: 0.6724
 ROC-AUC - epoch: 2 - score: 0.749803 

10/10 [==============================] - 9s 866ms/step - loss: 0.6334 - accuracy: 0.6724 - val_loss: 0.6376 - val_accuracy: 0.6375
Epoch 3/15
10/10 [==============================] - ETA: 0s - loss: 0.5916 - accuracy: 0.7618
 ROC-AUC - epoch: 3 - score: 0.740198 

10/10 [==============================] - 9s 878ms/step - loss: 0.5916 - accuracy: 0.7618 - val_loss: 0.6209 - val_accuracy: 0.6500
Epoch 4/15
10/10 [==============================] - ETA: 0s - loss: 0.5531 - accuracy: 0.7712
 ROC-AUC - epoch: 4 - score: 0.733900 

10/10 [==============================] - 9s 875ms/step - loss: 0.

OSError: ignored

In [ ]:
inp = Input(shape = (maxlen,))
x = Embedding(nb_words, embed_size, weights = [embedding_matrix], trainable = False)(inp)

x = SpatialDropout1D(dr)(x)

x = Bidirectional(GRU(units, return_sequences = True))(x)
x = Conv1D(64, kernel_size = 1, padding = "valid", kernel_initializer = "he_uniform")(x)
avg_pool = GlobalAveragePooling1D()(x)
max_pool = GlobalMaxPooling1D()(x)

x = concatenate([avg_pool, max_pool])
x = Dense(1, activation = "sigmoid")(x)
model = Model(inputs = [inp, body_text_flag], outputs = x)
ra_val = RocAucEvaluation(validation_data=([X_valid,body_flag_test], Y_valid), interval = 1)
model.compile(loss = "binary_crossentropy", optimizer = Adam(lr = lr, decay = lr_d), metrics = ["accuracy"])
history = model.fit(X_train , Y_train, batch_size = 64, epochs = 15, validation_data = (X_valid, Y_valid), 
                    verbose = 1, callbacks = [ra_val, check_point, early_stop])


In [ ]:
model = load_model(file_path)
model.load_weights(file_path)
predictions = model.predict([raw_text_test,body_flag_test]).reshape((raw_text_test.shape[0],))

# pred = loaded_model.predict(X_valid, batch_size = 1024, verbose = 1)
pred = np.where(predictions>=0.5,1,0)
score = pd.DataFrame(pred)
# print(Y_valid,score.astype(int))
accuracy_val = accuracy_score(Y_valid, score.values)
print('NN OOF accuracy: {}'.format(accuracy_val))